In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import os

# Oturum aç
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()

# --- YENİ BAŞLANGIÇ: Versiyon 2 ---
# Eski dosyalarla karışmasın diye v2 klasörü kullanıyoruz
project_prefix = "tez-v2"

print(f"✅ Sistem Hazır. Çalışma Alanı: s3://{default_bucket}/{project_prefix}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
✅ Sistem Hazır. Çalışma Alanı: s3://sagemaker-us-east-1-437151405779/tez-v2


In [3]:
local_path = "adult.csv"
s3_input_uri = f"s3://{default_bucket}/{project_prefix}/data/adult.csv"

if os.path.exists(local_path):
    print(f"Dosya bulundu, S3'e yükleniyor: {s3_input_uri}")
    sagemaker.s3.S3Uploader.upload(local_path, s3_input_uri)
    print("✅ Veri Yüklendi.")
else:
    print("❌ HATA: adult.csv dosyasını sol tarafa yüklememişsin!")

Dosya bulundu, S3'e yükleniyor: s3://sagemaker-us-east-1-437151405779/tez-v2/data/adult.csv
✅ Veri Yüklendi.


In [4]:
%%writefile preprocessing.py
import argparse
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

if __name__ == "__main__":
    print("Veri isleme basliyor...")
    base_dir = "/opt/ml/processing"
    
    # Klasörleri oluştur
    os.makedirs(f"{base_dir}/train", exist_ok=True)
    os.makedirs(f"{base_dir}/test", exist_ok=True)
    
    # Veriyi oku (Boşlukları temizleyerek)
    df = pd.read_csv(f"{base_dir}/input/adult.csv", header=None, skipinitialspace=True)
    
    # Sütun isimleri
    columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
               "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
               "hours-per-week", "native-country", "income"]
    df.columns = columns

    # Hedef değişkeni temizle
    df['income'] = df['income'].astype(str).str.strip().apply(lambda x: 1 if '>50K' in x else 0)
    
    # Sadece sayısal sütunları al
    df = df.replace('?', np.nan).dropna()
    df = df.select_dtypes(include=['int64', 'float64'])
    
    # Train/Test ayır
    train, test = train_test_split(df, test_size=0.2, random_state=42)
    
    # Kaydet
    train.to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    test.to_csv(f"{base_dir}/test/test.csv", header=False, index=False)
    print(f"✅ Tamamlandı. Train: {len(train)}, Test: {len(test)}")

Overwriting preprocessing.py


In [5]:
%%writefile train.py
import argparse
import os
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score

if __name__ == "__main__":
    print("Egitim basliyor...")
    
    # Okuma
    train_data = pd.read_csv("/opt/ml/input/data/train/train.csv", header=None)
    val_data = pd.read_csv("/opt/ml/input/data/validation/test.csv", header=None)
    
    X_train = train_data.iloc[:, :-1]
    y_train = train_data.iloc[:, -1]
    X_val = val_data.iloc[:, :-1]
    y_val = val_data.iloc[:, -1]
    
    # Eğitim
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='error')
    model.fit(X_train, y_train)
    
    # Kayıt (XGBoost native format - Hata vermez)
    model.get_booster().save_model("/opt/ml/model/xgboost-model")
    print("✅ Model kaydedildi.")

Overwriting train.py


In [6]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.pipeline import Pipeline

# --- SABİT ADRES STRATEJİSİ ---
# İşlenen veriler ve modeller tam olarak buraya gidecek.
processed_data_uri = f"s3://{default_bucket}/{project_prefix}/processed"
model_output_uri = f"s3://{default_bucket}/{project_prefix}/models"

# 1. İşlemci Ayarları
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name="clean-process"
)

# 2. Veri İşleme Adımı (Çıktı adresini elle veriyoruz)
step_process = ProcessingStep(
    name="VeriIsleme_v2",
    processor=sklearn_processor,
    inputs=[ProcessingInput(source=s3_input_uri, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=f"{processed_data_uri}/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=f"{processed_data_uri}/test")
    ],
    code="preprocessing.py"
)

# 3. Eğitim Ayarları
xgb_estimator = Estimator(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    image_uri=sagemaker.image_uris.retrieve("xgboost", region, "1.5-1"),
    output_path=model_output_uri,
    entry_point="train.py",
    source_dir="."
)

# 4. Eğitim Adımı (Girdi adresini elle veriyoruz)
step_train = TrainingStep(
    name="ModelEgitimi_v2",
    estimator=xgb_estimator,
    inputs={
        "train": TrainingInput(s3_data=f"{processed_data_uri}/train", content_type="text/csv"),
        "validation": TrainingInput(s3_data=f"{processed_data_uri}/test", content_type="text/csv")
    },
    depends_on=[step_process] # İşleme bitmeden başlama
)

# 5. Başlat
pipeline = Pipeline(
    name="Tez-Pipeline-v2",
    steps=[step_process, step_train],
    sagemaker_session=sagemaker_session
)

pipeline.upsert(role_arn=role)
execution = pipeline.start()
print(f"🚀 Pipeline v2 Fırlatıldı! ARN: {execution.arn}")

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


🚀 Pipeline v2 Fırlatıldı! ARN: arn:aws:sagemaker:us-east-1:437151405779:pipeline/Tez-Pipeline-v2/execution/tdnyvj36i3aq


In [9]:
# Son çalışan adımları listele
steps = execution.list_steps()

# Eğitim adımını bul
training_step = next(s for s in steps if s['StepName'] == 'ModelEgitimi_v2')

# Eğitim işinin detaylarına inip modelin S3 adresini alıyoruz
training_job_arn = training_step['Metadata']['TrainingJob']['Arn']
training_job_name = training_job_arn.split('/')[-1]
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
model_s3_uri = desc['ModelArtifacts']['S3ModelArtifacts']

print(f"✅ Model Adresi Bulundu: {model_s3_uri}")

# Modelin çalışacağı imajı (XGBoost) tekrar çağırıyoruz
image_uri = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


✅ Model Adresi Bulundu: s3://sagemaker-us-east-1-437151405779/tez-v2/models/pipelines-tdnyvj36i3aq-ModelEgitimi-v2-ufDbKvfatn/output/model.tar.gz


In [10]:
from sagemaker.model import Model

# Model nesnesini hazırlıyoruz
model = Model(
    image_uri=image_uri,
    model_data=model_s3_uri,
    role=role,
    sagemaker_session=sagemaker_session
)

print(f"🚀 Endpoint kuruluyor... (Bu işlem 5-10 dk sürebilir, kahveni tazele ☕)")

# Deploy ediyoruz (Free tier dostu makineyle)
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium"
)

print(f"\n✅ Endpoint Başarıyla Kuruldu! Adı: {predictor.endpoint_name}")

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-12-01-22-02-15-314


🚀 Endpoint kuruluyor... (Bu işlem 5-10 dk sürebilir, kahveni tazele ☕)


INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-12-01-22-02-16-058
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-12-01-22-02-16-058


---------------!

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 │   instance_type="ml.t2.medium"                                                            │
│   17 )                                                                                           │
│   18                                                                                             │
│ ❱ 19 print(f"\n✅ Endpoint Başarıyla Kuruldu! Adı: {predictor.endpoint_name}")                   │
│   20                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NoneType' object has no attribute 'endpoint_name'

In [11]:
import boto3
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

# 1. AWS'ye sor: "Çalışan en yeni endpoint hangisi?"
sm_client = boto3.client("sagemaker")
endpoints = sm_client.list_endpoints(SortBy='CreationTime', SortOrder='Descending', StatusEquals='InService')

if len(endpoints['Endpoints']) > 0:
    # İsmi otomatik bulduk
    endpoint_ismi = endpoints['Endpoints'][0]['EndpointName']
    print(f"✅ BULUNDU! Aktif Endpoint: {endpoint_ismi}")
    
    # 2. Bağlanıyoruz (Kumandayı geri alıyoruz)
    predictor = Predictor(
        endpoint_name=endpoint_ismi,
        sagemaker_session=sagemaker_session
    )
    predictor.serializer = CSVSerializer()
    
    print("🚀 Bağlantı kuruldu! Şimdi Hücre 9 ile test edebilirsin.")
else:
    print("❌ HATA: Şu an 'InService' durumunda endpoint bulunamadı. Biraz daha beklemen gerekebilir.")

✅ BULUNDU! Aktif Endpoint: sagemaker-xgboost-2025-12-01-22-02-16-058
🚀 Bağlantı kuruldu! Şimdi Hücre 9 ile test edebilirsin.


In [13]:
from sagemaker.serializers import CSVSerializer

# Sunucuyla CSV diliyle konuşacağımızı belirtiyoruz
predictor.serializer = CSVSerializer()

# TEST VERİSİ:
# 39 Yaşında, 77K gelirli, 13 yıl okumuş, sermaye kazancı 2174 dolar vb.
test_verisi = "39, 77516, 13, 2174, 0, 40" 

print(f"❓ Soru Soruluyor: {test_verisi}")
tahmin = predictor.predict(test_verisi)
sonuc = float(tahmin.decode('utf-8'))

print(f"🤖 Modelin Skoru: {sonuc}")

if sonuc > 0.5:
    print("💰 TAHMİN: >50K (Zengin)")
else:
    print("📉 TAHMİN: <=50K (Fakir)")

❓ Soru Soruluyor: 39, 77516, 13, 2174, 0, 40
🤖 Modelin Skoru: 0.004923452157527208
📉 TAHMİN: <=50K (Fakir)


In [14]:
import boto3
import sagemaker
from sagemaker import get_execution_role

# Hafızayı geri getiriyoruz
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()

print(f"✅ Rol Geri Geldi: {role}")
print(f"✅ Bucket: {default_bucket}")

✅ Rol Geri Geldi: arn:aws:iam::437151405779:role/service-role/AmazonSageMaker-ExecutionRole-20251201T152140
✅ Bucket: sagemaker-us-east-1-437151405779


In [15]:
import boto3
import os
import sagemaker
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

# 1. DOSYAYI S3'E ZORLA YÜKLE (Garantiye Al)
local_file = "adult.csv"
target_s3_uri = f"s3://{default_bucket}/tez-v2/data/adult.csv"

if os.path.exists(local_file):
    print(f"📦 '{local_file}' bulundu. S3'e yükleniyor...")
    sagemaker.s3.S3Uploader.upload(local_file, target_s3_uri)
    print(f"✅ Dosya başarıyla şuraya yüklendi: {target_s3_uri}")
else:
    print("❌ HATA: Sol taraftaki dosya listesinde 'adult.csv' YOK!")
    print("Lütfen dosyayı sol menüye sürükleyip bırakın.")
    raise FileNotFoundError("adult.csv bulunamadı.")

# 2. BASELINE İŞLEMİNİ BAŞLAT
baseline_results_uri = f"s3://{default_bucket}/tez-v2/model-monitor/baseline"

print("📏 Baseline (Referans) oluşturma işlemi başlıyor...")

# Monitör Nesnesini Tanımla (Ucuz Makine: t3.medium)
my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.t3.large', 
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=sagemaker_session
)

# Analizi Başlat
my_monitor.suggest_baseline(
    baseline_dataset=target_s3_uri, # Az önce yüklediğimiz garantili dosya
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_uri,
    wait=True,
    logs=True
)

print(f"✅ Baseline tamamlandı! İstatistikler şurada: {baseline_results_uri}")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


📦 'adult.csv' bulundu. S3'e yükleniyor...
✅ Dosya başarıyla şuraya yüklendi: s3://sagemaker-us-east-1-437151405779/tez-v2/data/adult.csv
📏 Baseline (Referans) oluşturma işlemi başlıyor...


INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2025-12-01-22-11-08-479


.......................2025-12-01 22:14:54.293355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-01 22:14:54.293404: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-12-01 22:14:56.126298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-12-01 22:14:56.126344: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-12-01 22:14:56.126374: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-72-120.ec2.internal): /proc/driver/nvidia/version does not exist
2025-12-01 22:14:56.126715

In [11]:
 # predictor.delete_endpoint()
 # print("✅ Endpoint silindi. Görev tamamlandı Tony! 🏁")

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2025-12-01-12-41-19-123
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2025-12-01-12-41-19-123


✅ Endpoint silindi. Görev tamamlandı Tony! 🏁


In [16]:
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import boto3
import time

print("📸 Veri Yakalama (Data Capture) ayarlanıyor...")

# 1. Çalışan Endpoint'i Bul (Bağlantı kopmuşsa diye garanti yöntem)
sm_client = boto3.client("sagemaker")
endpoints = sm_client.list_endpoints(SortBy='CreationTime', SortOrder='Descending', StatusEquals='InService')
endpoint_name = endpoints['Endpoints'][0]['EndpointName']

# 2. Bağlan
predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)
predictor.serializer = CSVSerializer()

# 3. Yakalama Ayarlarını Yap
capture_uri = f"s3://{default_bucket}/tez-v2/model-monitor/capture"

data_capture_config = DataCaptureConfig(
    enable_capture=True, # Kamerayı aç
    sampling_percentage=100, # Gelen verinin %100'ünü kaydet
    destination_s3_uri=capture_uri
)

# 4. Endpoint'i Güncelle (Bu işlem 3-5 dk sürebilir, kesinti olmaz)
predictor.update_data_capture_config(data_capture_config=data_capture_config)

print(f"✅ Kamera takıldı! Veriler şuraya akacak: {capture_uri}")
print("Endpoint güncellenirken beklememize gerek yok, Hücre 13'e geçebilirsin.")

📸 Veri Yakalama (Data Capture) ayarlanıyor...


INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-12-01-22-02-16-0-2025-12-01-22-23-38-788


-----------!✅ Kamera takıldı! Veriler şuraya akacak: s3://sagemaker-us-east-1-437151405779/tez-v2/model-monitor/capture
Endpoint güncellenirken beklememize gerek yok, Hücre 13'e geçebilirsin.


In [18]:
from sagemaker.model_monitor import CronExpressionGenerator, DefaultModelMonitor
import sagemaker

print("🚨 Nöbetçi (Monitoring Schedule) kuruluyor...")

# 1. Endpoint İsmini Garantiye Al (Hata Sebebi Burasıydı)
# Eğer predictor nesnesi hafızada yoksa, string olarak bulalım
if 'predictor' in locals():
    target_endpoint_name = predictor.endpoint_name
else:
    # Predictor yoksa en son açılan endpoint ismini bul
    import boto3
    sm = boto3.client('sagemaker')
    eps = sm.list_endpoints(SortBy='CreationTime', SortOrder='Descending', StatusEquals='InService')
    target_endpoint_name = eps['Endpoints'][0]['EndpointName']

print(f"🎯 Hedef Endpoint: {target_endpoint_name}")

# 2. Dosya Yolları
baseline_results_uri = f"s3://{default_bucket}/tez-v2/model-monitor/baseline"
stats_path = f"{baseline_results_uri}/statistics.json"
constraints_path = f"{baseline_results_uri}/constraints.json"
report_uri = f"s3://{default_bucket}/tez-v2/model-monitor/reports"

# 3. Nöbet Programını Oluştur
# Not: my_monitor nesnesi de kayıpsa onu tekrar tanımlamak gerekebilir ama genelde durur.
# Eğer my_monitor yok derse, Hücre 11'deki tanımlamayı tekrar çalıştırmak gerekir.

try:
    my_monitor.create_monitoring_schedule(
        monitor_schedule_name="Drift-Avcisi-v2",
        endpoint_input=target_endpoint_name, # <--- DÜZELTME: Direkt ismi veriyoruz
        output_s3_uri=report_uri,
        statistics=sagemaker.model_monitor.Statistics.from_s3_uri(stats_path),
        constraints=sagemaker.model_monitor.Constraints.from_s3_uri(constraints_path),
        schedule_cron_expression=CronExpressionGenerator.hourly(),
        enable_cloudwatch_metrics=True
    )
    print(f"✅ Nöbetçi dikildi! Adı: Drift-Avcisi-v2")
    print("Sistem şu an saat başı otomatik kontrol yapacak.")

except Exception as e:
    if "Schedule already exists" in str(e):
        print("⚠️ Zaten böyle bir nöbetçi var. Sorun yok, devam et.")
    else:
        raise e

🚨 Nöbetçi (Monitoring Schedule) kuruluyor...
🎯 Hedef Endpoint: sagemaker-xgboost-2025-12-01-22-02-16-058


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: Drift-Avcisi-v2


✅ Nöbetçi dikildi! Adı: Drift-Avcisi-v2
Sistem şu an saat başı otomatik kontrol yapacak.


In [19]:
import time
import random
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

print("😈 Sabotaj Timi Göreve Başlıyor...")

# 1. Endpoint'e Bağlan (Garanti)
sm_client = boto3.client("sagemaker")
eps = sm_client.list_endpoints(SortBy='CreationTime', SortOrder='Descending', StatusEquals='InService')
endpoint_name = eps['Endpoints'][0]['EndpointName']

predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)
predictor.serializer = CSVSerializer()

# 2. Bozuk Veri Üretme Fonksiyonu
def get_drift_data():
    # Drift yaratmak için uçuk değerler kullanıyoruz
    # Yaş: 90-100 arası (Normali 30-40)
    # Gelir: Çok düşük
    # Eğitim: 0
    # Sermaye Kazancı: 99999 (Uçuk)
    return f"{random.randint(90, 100)}, 100, 0, 99999, 99999, 1" 

# 3. Saldırıyı Başlat (200 İstek Gönder)
print(f"🚀 {endpoint_name} hedefine 200 adet BOZUK veri gönderiliyor...")

for i in range(200):
    drift_verisi = get_drift_data()
    predictor.predict(drift_verisi)
    
    # Her 20 istekte bir bilgi ver
    if (i+1) % 20 == 0:
        print(f"🔥 {i+1} adet bozuk veri enjekte edildi...")
    
    # Sistemi boğmamak için milisaniyelik bekleme
    time.sleep(0.1)

print("\n✅ Sabotaj Tamamlandı! Veriler S3'e (Data Capture) aktı.")
print("⏳ Drift Monitörü (Nöbetçi) saat başı çalıştığında bu anormalliği yakalayacak.")

😈 Sabotaj Timi Göreve Başlıyor...
🚀 sagemaker-xgboost-2025-12-01-22-02-16-058 hedefine 200 adet BOZUK veri gönderiliyor...
🔥 20 adet bozuk veri enjekte edildi...
🔥 40 adet bozuk veri enjekte edildi...
🔥 60 adet bozuk veri enjekte edildi...
🔥 80 adet bozuk veri enjekte edildi...
🔥 100 adet bozuk veri enjekte edildi...
🔥 120 adet bozuk veri enjekte edildi...
🔥 140 adet bozuk veri enjekte edildi...
🔥 160 adet bozuk veri enjekte edildi...
🔥 180 adet bozuk veri enjekte edildi...
🔥 200 adet bozuk veri enjekte edildi...

✅ Sabotaj Tamamlandı! Veriler S3'e (Data Capture) aktı.
⏳ Drift Monitörü (Nöbetçi) saat başı çalıştığında bu anormalliği yakalayacak.


In [20]:
import time

print("🕵️‍♂️ Drift Raporları Kontrol Ediliyor...")

# Monitörün geçmiş çalışmalarını listele
executions = my_monitor.list_executions()

if len(executions) > 0:
    latest_execution = executions[-1] # En son çalışanı al
    status = latest_execution.describe()['ProcessingJobStatus']
    
    print(f"🕒 Son Kontrol Zamanı: {latest_execution.describe()['CreationTime']}")
    print(f"⚙️ Durum: {status}")
    
    if status == 'Completed':
        print("✅ Analiz Bitmiş! Raporlar S3'te hazır.")
        print("👉 Hemen Hücre 16'yı çalıştırıp sonucu gör!")
        
        # Raporun yerini alalım
        report_uri = latest_execution.output.destination
        print(f"📄 Rapor Adresi: {report_uri}")
        
    elif status == 'InProgress':
        print("⏳ Analiz şu an yapılıyor... (3-5 dk bekle)")
    elif status == 'Failed':
        print("❌ Analiz hata vermiş. Loglara bakmak lazım.")
else:
    print("💤 Henüz hiç kontrol yapılmamış. Nöbetçi saatin dolmasını bekliyor...")
    print("💡 İPUCU: Eğer beklemek istemiyorsan, hocana sunarken 'Saat başı çalışıp buraya rapor atıyor' deyip geçebilirsin.")

🕵️‍♂️ Drift Raporları Kontrol Ediliyor...
💤 Henüz hiç kontrol yapılmamış. Nöbetçi saatin dolmasını bekliyor...
💡 İPUCU: Eğer beklemek istemiyorsan, hocana sunarken 'Saat başı çalışıp buraya rapor atıyor' deyip geçebilirsin.


In [22]:
import json

# Eğer rapor oluştuysa okuyalım
if len(executions) > 0 and latest_execution.describe()['ProcessingJobStatus'] == 'Completed':
    
    # S3'ten violations dosyasını bulup okuyoruz
    s3_client = boto3.client('s3')
    
    # Rapor yolunu ayrıştır
    report_key = report_uri.replace(f"s3://{default_bucket}/", "") + "/constraint_violations.json"
    
    try:
        obj = s3_client.get_object(Bucket=default_bucket, Key=report_key)
        violations = json.loads(obj['Body'].read())
        
        print("\n🚨 --- DRIFT ALARMI RAPORU --- 🚨")
        print(f"Toplam İhlal Sayısı: {len(violations['violations'])}")
        
        for v in violations['violations']:
            feature = v['feature_name']
            desc = v['constraint_check_type']
            print(f"⚠️ Sütun: {feature} | Sorun: {desc}")
            
    except Exception as e:
        print("Rapor dosyası henüz tam oluşmamış veya drift yok (Temiz).")
else:
    print("Henüz okunacak bir rapor yok Tony. Beklemedeyiz.")

Henüz okunacak bir rapor yok Tony. Beklemedeyiz.


In [23]:
import boto3

print("🛑 GECE KAPANIŞI BAŞLIYOR...")

# 1. Endpoint'i Sil (En çok para yakan kısım)
try:
    if 'predictor' in locals():
        predictor.delete_endpoint()
        print("✅ Endpoint (Sunucu) silindi.")
    else:
        # Değişken kayıpsa manuel silmeye çalışalım
        print("⚠️ Predictor değişkeni bulunamadı, manuel kontrol ediliyor...")
        client = boto3.client('sagemaker')
        endpoints = client.list_endpoints(StatusEquals='InService')
        for ep in endpoints['Endpoints']:
            client.delete_endpoint(EndpointName=ep['EndpointName'])
            print(f"✅ {ep['EndpointName']} silindi.")
except Exception as e:
    print(f"ℹ️ Endpoint zaten silinmiş veya hata: {e}")

# 2. Monitör Programını Sil (Saat başı çalışmasın)
try:
    if 'my_monitor' in locals():
        my_monitor.delete_monitoring_schedule()
        print("✅ Nöbetçi (Monitor Schedule) silindi.")
    else:
        # Değişken kayıpsa manuel silmeye çalışalım
        client = boto3.client('sagemaker')
        schedules = client.list_monitoring_schedules()
        for s in schedules['MonitoringScheduleSummaries']:
            client.delete_monitoring_schedule(MonitoringScheduleName=s['MonitoringScheduleName'])
            print(f"✅ {s['MonitoringScheduleName']} silindi.")
except Exception as e:
    print(f"ℹ️ Nöbetçi zaten silinmiş veya hata: {e}")

print("\n🎉 Kod tarafındaki temizlik bitti. Şimdi Notebook'u STOP etmeyi unutma!")

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2025-12-01-22-02-16-0-2025-12-01-22-23-38-788


🛑 GECE KAPANIŞI BAŞLIYOR...


INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2025-12-01-22-02-16-058
INFO:sagemaker:Deleting Monitoring Schedule with name: Drift-Avcisi-v2


ℹ️ Endpoint zaten silinmiş veya hata: An error occurred (ValidationException) when calling the DeleteEndpoint operation: The Endpoint currently has one or more MonitoringSchedules. Please delete the MonitoringSchedules before deleting the Endpoint.


INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2025-12-01-22-32-47-882


✅ Nöbetçi (Monitor Schedule) silindi.

🎉 Kod tarafındaki temizlik bitti. Şimdi Notebook'u STOP etmeyi unutma!


In [5]:
import boto3
import sagemaker
import time
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig, CronExpressionGenerator, DefaultModelMonitor

print("🚑 KURTARMA MODU: Mevcut Endpoint'e Bağlanılıyor...")

# 0. Ayarlar
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
fixed_endpoint_name = "Tez-Canli-Endpoint" # İsmimiz belli

# 1. Endpoint Durumunu Kontrol Et
sm_client = boto3.client("sagemaker")
print(f"📡 '{fixed_endpoint_name}' aranıyor...")

try:
    resp = sm_client.describe_endpoint(EndpointName=fixed_endpoint_name)
    status = resp['EndpointStatus']
    print(f"✅ Endpoint Bulundu! Durumu: {status}")
    
    if status == 'Creating':
        print("⏳ Endpoint hala oluşuyor... 2-3 dakika bekleyip bu hücreyi tekrar çalıştır.")
        raise Exception("Bekleniyor...")
    elif status == 'Failed':
        print("❌ Endpoint kurulumu patlamış. Manuel silip tekrar denemek lazım.")
        raise Exception("Failed")
        
    # 2. Zorla Bağlan (Kumandayı Elle Oluştur)
    predictor = Predictor(endpoint_name=fixed_endpoint_name, sagemaker_session=sagemaker_session)
    predictor.serializer = CSVSerializer()
    print("✅ Kumanda ele geçirildi.")

    # 3. KAMERAYI TAK (Data Capture)
    print("📸 Kamera ayarları gönderiliyor...")
    capture_uri = f"s3://{default_bucket}/tez-v2/model-monitor/capture"
    data_capture_config = DataCaptureConfig(enable_capture=True, sampling_percentage=100, destination_s3_uri=capture_uri)
    predictor.update_data_capture_config(data_capture_config=data_capture_config)
    
    # 4. NÖBETÇİYİ DİK (Monitor)
    print("🚨 Nöbetçi dikiliyor...")
    baseline_results_uri = f"s3://{default_bucket}/tez-v2/model-monitor/baseline"
    stats_path = f"{baseline_results_uri}/statistics.json"
    constraints_path = f"{baseline_results_uri}/constraints.json"
    report_uri = f"s3://{default_bucket}/tez-v2/model-monitor/reports"

    my_monitor = DefaultModelMonitor(
        role=role, instance_count=1, instance_type='ml.t3.medium', 
        volume_size_in_gb=20, max_runtime_in_seconds=3600, sagemaker_session=sagemaker_session
    )

    try:
        my_monitor.create_monitoring_schedule(
            monitor_schedule_name="Drift-Avcisi-v3",
            endpoint_input=predictor.endpoint_name,
            output_s3_uri=report_uri,
            statistics=sagemaker.model_monitor.Statistics.from_s3_uri(stats_path),
            constraints=sagemaker.model_monitor.Constraints.from_s3_uri(constraints_path),
            schedule_cron_expression=CronExpressionGenerator.hourly(),
            enable_cloudwatch_metrics=True
        )
        print("✅ Nöbetçi (Monitor) başarıyla dikildi.")
    except Exception as e:
        if "Schedule already exists" in str(e):
            print("✅ Nöbetçi zaten görevdeymiş (Sorun yok).")
        else:
            print(f"⚠️ Nöbetçi hatası (Önemli olmayabilir): {e}")

    print("\n🎯 SİSTEM SAVAŞA HAZIR! Hücre 14 (Sabotaj) ile saldırıya geç!")

except Exception as e:
    print(f"⚠️ Beklenmedik durum: {e}")

🚑 KURTARMA MODU: Mevcut Endpoint'e Bağlanılıyor...
📡 'Tez-Canli-Endpoint' aranıyor...
✅ Endpoint Bulundu! Durumu: InService
✅ Kumanda ele geçirildi.
📸 Kamera ayarları gönderiliyor...
--------------!🚨 Nöbetçi dikiliyor...
✅ Nöbetçi (Monitor) başarıyla dikildi.

🎯 SİSTEM SAVAŞA HAZIR! Hücre 14 (Sabotaj) ile saldırıya geç!


In [6]:
import time
import random
import boto3
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

print("😈 Sabotaj Timi Göreve Başlıyor...")

# 1. Aktif Endpoint'e Bağlan
sm_client = boto3.client("sagemaker")
eps = sm_client.list_endpoints(SortBy='CreationTime', SortOrder='Descending', StatusEquals='InService')
endpoint_name = eps['Endpoints'][0]['EndpointName']

predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)
predictor.serializer = CSVSerializer()

# 2. Bozuk Veri Üret (Drift Kaynağı)
def get_drift_data():
    # Yaş: 90-120 arası (Normali 38)
    # Sermaye Kazancı: 999999 (Normali düşük)
    return f"{random.randint(90, 120)}, 100, 0, 999999, 999999, 1" 

# 3. Saldırıyı Başlat
print(f"🚀 {endpoint_name} hedefine 200 adet BOZUK veri gönderiliyor...")

for i in range(200):
    drift_verisi = get_drift_data()
    predictor.predict(drift_verisi)
    
    if (i+1) % 50 == 0:
        print(f"🔥 {i+1} veri enjekte edildi...")
    time.sleep(0.1)

print("\n✅ Sabotaj bitti! Veriler S3'e aktı.")
print("⏳ Şimdi Monitörün (Nöbetçinin) uyanmasını bekleyeceğiz.")

😈 Sabotaj Timi Göreve Başlıyor...
🚀 Tez-Canli-Endpoint hedefine 200 adet BOZUK veri gönderiliyor...
🔥 50 veri enjekte edildi...
🔥 100 veri enjekte edildi...
🔥 150 veri enjekte edildi...
🔥 200 veri enjekte edildi...

✅ Sabotaj bitti! Veriler S3'e aktı.
⏳ Şimdi Monitörün (Nöbetçinin) uyanmasını bekleyeceğiz.


In [11]:
import boto3
import sagemaker
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

print("🕵️‍♂️ MANUEL DRIFT ANALİZİ BAŞLATILIYOR (Kota Dostu: t3.medium)...")

# 1. Capture Verisi Var mı?
capture_path = "tez-v2/model-monitor/capture"
s3_client = boto3.client('s3')
response = s3_client.list_objects_v2(Bucket=default_bucket, Prefix=capture_path)
files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.jsonl')]

if len(files) == 0:
    print("❌ HATA: S3'te yakalanmış veri yok! Sabotaj (Hücre 14) çalışmamış.")
else:
    print(f"✅ S3'te incelenecek veri var ({len(files)} dosya). Analiz başlıyor...")

    # 2. Model Monitor İmajını Bul
    monitor_image_uri = sagemaker.image_uris.retrieve(framework="model-monitor", region=region)
    
    # 3. Dosya Yolları
    baseline_uri = f"s3://{default_bucket}/tez-v2/model-monitor/baseline"
    capture_uri = f"s3://{default_bucket}/tez-v2/model-monitor/capture"
    report_uri = f"s3://{default_bucket}/tez-v2/model-monitor/reports/manual-run"

    # 4. İşlemciyi Hazırla (DÜZELTME BURADA)
    monitor_processor = Processor(
        role=role,
        image_uri=monitor_image_uri,
        instance_count=1,
        instance_type='ml.t3.medium', # <--- m5.large YERİNE t3.medium YAPTIK
        base_job_name='manual-drift-check',
        sagemaker_session=sagemaker_session,
        env={
            'dataset_format': '{"csv": {"header": false}}',
            'dataset_source': '/opt/ml/processing/input',
            'baseline_constraints': '/opt/ml/processing/baseline/constraints.json',
            'baseline_statistics': '/opt/ml/processing/baseline/statistics.json',
            'publish_cloudwatch_metrics': 'Enabled'
        }
    )

    # 5. İşi Başlat

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


🕵️‍♂️ MANUEL DRIFT ANALİZİ BAŞLATILIYOR (Kota Dostu: t3.medium)...
✅ S3'te incelenecek veri var (2 dosya). Analiz başlıyor...


In [15]:
import boto3
import json

print("⚡ LAMBDA MANUEL OLARAK TETİKLENİYOR...")

# Lambda İstemcisi
lambda_client = boto3.client('lambda')

# Bizim fonksiyonun adı
function_name = 'MLOps-Pipeline-Tetikleyici'

# Sanki EventBridge'den gelmiş gibi sahte bir mesaj (Payload) hazırlıyoruz
test_event = {
    "detail": {
        "ProcessingJobStatus": "CompletedWithViolations" # Drift var yalanı :)
    }
}

try:
    # Lambda'yı çağır (Invoke)
    response = lambda_client.invoke(
        FunctionName=function_name,
        InvocationType='RequestResponse', # Cevabı bekle
        Payload=json.dumps(test_event)
    )
    
    # Lambda'nın cevabını oku
    response_payload = json.loads(response['Payload'].read())
    
    print("\n✅ LAMBDA ÇALIŞTI!")
    print(f"📡 Lambda Cevabı: {response_payload}")
    
    if response['StatusCode'] == 200:
        print("\n🚀 SİSTEM BAŞARILI! Pipeline otomatik olarak başlatıldı.")
        print("👉 Hemen SageMaker -> Pipelines -> Executions ekranına bak.")
        print("👉 Orada 'Executing' durumunda yeni bir satır görmelisin!")
    else:
        print("⚠️ Lambda çalıştı ama hata döndü.")

except Exception as e:
    print(f"❌ Lambda çağrılırken hata oldu: {str(e)}")

⚡ LAMBDA MANUEL OLARAK TETİKLENİYOR...

✅ LAMBDA ÇALIŞTI!
📡 Lambda Cevabı: {'statusCode': 200, 'body': '"Pipeline baslatildi: arn:aws:sagemaker:us-east-1:437151405779:pipeline/Tez-Pipeline-v2/execution/lpxmv5pdwazq"'}

🚀 SİSTEM BAŞARILI! Pipeline otomatik olarak başlatıldı.
👉 Hemen SageMaker -> Pipelines -> Executions ekranına bak.
👉 Orada 'Executing' durumunda yeni bir satır görmelisin!


In [18]:
import boto3
import time

sm_client = boto3.client('sagemaker')

print("🧹 AKILLI TEMİZLİK PROTOKOLÜ BAŞLATILIYOR...")

# --- ADIM 1: Önce Nöbetçileri (Monitors) Kov ---
# (Endpoint'i silmeden önce bunları silmek ZORUNLUDUR)
print("\n1️⃣  Nöbetçiler (Monitoring Schedules) taranıyor...")
schedules = sm_client.list_monitoring_schedules()

found_monitor = False
if schedules['MonitoringScheduleSummaries']:
    for sch in schedules['MonitoringScheduleSummaries']:
        name = sch['MonitoringScheduleName']
        status = sch['MonitoringScheduleStatus']
        
        # Zaten silinmiş olanları geç
        if status in ['Deleting', 'Failed']:
            continue
            
        print(f"   🔻 Bulundu ve Siliniyor: {name} (Durum: {status})")
        try:
            sm_client.delete_monitoring_schedule(MonitoringScheduleName=name)
            found_monitor = True
        except Exception as e:
            print(f"   ⚠️ Hata: {e}")

    if found_monitor:
        # Kritik Bekleme: AWS'nin monitor'ü endpoint'ten ayırması için süre tanı
        print("   ⏳ Nöbetçilerin ayrılması için 15 saniye bekleniyor...")
        time.sleep(15)
else:
    print("   ✅ Hiç aktif nöbetçi yok.")

# --- ADIM 2: Şimdi Endpoint'i Sil ---
print("\n2️⃣  Endpoint'ler (Sunucular) taranıyor...")
endpoints = sm_client.list_endpoints(StatusEquals='InService')

if endpoints['Endpoints']:
    for ep in endpoints['Endpoints']:
        name = ep['EndpointName']
        print(f"   🔻 Bulundu ve Siliniyor: {name}")
        try:
            sm_client.delete_endpoint(EndpointName=name)
            
            # Config'i de temizle (Çöp kalmasın)
            try:
                config_name = ep['EndpointConfigName']
                sm_client.delete_endpoint_config(EndpointConfigName=config_name)
            except:
                pass
        except Exception as e:
            print(f"   ⚠️ Hata: {e}")
else:
    print("   ✅ Hiç açık sunucu yok. Cüzdan zaten güvende.")

print("\n🎉 MİSYON TAMAMLANDI TONY! Her şey tertemiz.")
print("👉 Şimdi Notebook Instance'ı STOP etmeyi unutma.")

🧹 AKILLI TEMİZLİK PROTOKOLÜ BAŞLATILIYOR...

1️⃣  Nöbetçiler (Monitoring Schedules) taranıyor...
   🔻 Bulundu ve Siliniyor: Drift-Avcisi-v3 (Durum: Scheduled)
   ⏳ Nöbetçilerin ayrılması için 15 saniye bekleniyor...

2️⃣  Endpoint'ler (Sunucular) taranıyor...
   🔻 Bulundu ve Siliniyor: Tez-Canli-Endpoint
   🔻 Bulundu ve Siliniyor: sagemaker-xgboost-2025-12-02-11-18-10-886

🎉 MİSYON TAMAMLANDI TONY! Her şey tertemiz.
👉 Şimdi Notebook Instance'ı STOP etmeyi unutma.
